In [3]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import cv2

import napari

In [5]:
# Change the input image directory in functions crop_to_fit() and crop_to_fit_dilate() below

def get_pixel_coordinates(mask, cell_number):
    """Get the coordinates of all pixels with a specific cell number in the mask.
    Z-coordinates are scaled by 5.8824, the resampling factor for isotropic voxels."""
    
    coords = np.argwhere(mask == cell_number)
    coords[:, 0] = (coords[:, 0] / 5.8824).astype(int)

    return coords


def expand_coordinates(coords, expansion_range):
    """
    Expands the range of x and y coordinates by a specified range.
    
    Args:
        coords (numpy.ndarray): Array of shape (N, 3) representing the z, y, x coordinates.
        expansion_range (int): Number of pixels to expand in both directions for x and y.
    
    Returns:
        numpy.ndarray: Expanded coordinates.
    """
    # Extract z, y, x coordinates
    z_coords, y_coords, x_coords = coords[:, 0], coords[:, 1], coords[:, 2]
    
    # Calculate the min and max for x and y
    y_min, y_max = np.min(y_coords), np.max(y_coords)
    x_min, x_max = np.min(x_coords), np.max(x_coords)
    
    # Expand the range
    y_min_exp, y_max_exp = y_min - expansion_range, y_max + expansion_range
    x_min_exp, x_max_exp = x_min - expansion_range, x_max + expansion_range

    # Clip values to ensure they stay within the image bounds
    y_min_exp = max(0, y_min_exp)
    x_min_exp = max(0, x_min_exp)
    y_max_exp = min(y_max_exp, 2303)
    x_max_exp = min(x_max_exp, 2303)
    
    # Create the new expanded range of coordinates
    expanded_coords = []
    for z in np.unique(z_coords):
        for y in range(y_min_exp, y_max_exp + 1):
            for x in range(x_min_exp, x_max_exp + 1):
                expanded_coords.append((z, y, x))
    
    return np.array(expanded_coords)


    
def crop_to_fit(mask, coords, x, time_point, channel,cell_number):
    # extracts a tight 3D crop of a masked region from a large z-stack image
    
    img_dir = "./Bigstream_notebook/"
    aligned_img_name = "Aligned_zstack"+ str(x) + '_Time' + str(time_point)+ '_channel'+str(channel)+ '.tiff'
    aligned_img_tif = os.path.join(img_dir, aligned_img_name)
    img = tifffile.imread(aligned_img_tif)

    z_min, y_min, x_min = coords.min(axis=0)   # load voxel coordinates of a mask
    z_max, y_max, x_max = coords.max(axis=0)

    # Create an empty output image with the size of the cropped image
    cropped_shape = (z_max - z_min + 1, y_max - y_min + 1, x_max - x_min + 1)
    cropped_image = np.zeros(cropped_shape, dtype=mask.dtype)

    # Populate the cropped image with values from the original mask
    for z, y, x in coords:
        cropped_image[z - z_min, y - y_min, x - x_min] = img[z, y, x]
    
    return cropped_image

def crop_to_fit_dilate(mask, coords, x, time_point, channel,cell_number, padding):
    img_dir = "./Bigstream_notebook/"
    aligned_img_name = "Aligned_zstack"+ str(x) + '_Time' + str(time_point)+ '_channel'+str(channel)+ '.tiff'
    aligned_img_tif = os.path.join(img_dir, aligned_img_name)
    img = tifffile.imread(aligned_img_tif)

    z_min, y_min, x_min = coords.min(axis=0)
    z_max, y_max, x_max = coords.max(axis=0)
    
    #dilate in x, y
    y_min = y_min - padding
    x_min = x_min - padding
    y_max = y_max + padding
    x_max = x_max + padding
    
    # Determine the size of the new, smaller cropped image
    cropped_shape = (z_max - z_min + 1, y_max - y_min + 1, x_max - x_min + 1)
    cropped_image = np.zeros(cropped_shape, dtype=mask.dtype)

    # Populate the cropped image with values from the original mask
    for z, y, x in coords:
        cropped_image[z - z_min, y - y_min, x - x_min] = img[z, y, x]

    middle_z = int((z_max - z_min)/2)
    cropped_image = cropped_image[middle_z,:,:]
    return cropped_image



def neuron_or_not(cropped_image, threshold):    # threshold is the cut-off of average pixel intensity
    """Evaluate NeuN average intensity for identifying neurons.
    Calculate the mean pixel value of the non-zero pixels in the cropped image."""
    
    non_zero_pixels = cropped_image[cropped_image > 0]  # Only consider non-zero pixels
    mean_value = np.mean(non_zero_pixels)
    if mean_value > threshold:
        return True
    else:
        return False


def glial_or_not_n_pixels(cropped_image, threshold):   
    # evaluate GFAP intensity for identifying astrocyte. Threshold is the cut-off of pixel intensity
    
    pixels = cropped_image[cropped_image > threshold]  # count pixels higher than threshold value
    n_pixels = len(pixels)
    if n_pixels > 3000:            # more than 3000 pixels of GFAP signal
        return True
    else:
        return False

def save_nuclei(cropped_image, outdir, x, time_point, channel, cell_number):
    z_folder = 'z'+str(x)+'/'
    if not os.path.exists(os.path.join(outdir,z_folder)):
        os.makedirs(os.path.join(outdir,z_folder))
    outfile_prefix = 'z'+str(x)+'/t'+str(time_point)+'_c'+str(channel)+'_cell'+str(cell_number)
    outfile_tiff = os.path.join(outdir,outfile_prefix+'.tiff')
    tifffile.imwrite(outfile_tiff,cropped_image)

    return cropped_image

In [9]:
padding = 5   # diameter of ring surrounding nuclear mask for GFAP intensity evaluation

for x in range(0,41):   # 0-41 for 40 images, each for a field of view
    print(x)
    cell_list = []
    cell_type_list = []
    # z_series_list = []

    mask_dir = "./CP_masks4/"   # Folder of Cellpose mask images
    mask_name = "z"+ str(x) + '.tif'
    mask_tiff = os.path.join(mask_dir, mask_name)
    mask = tifffile.imread(mask_tiff)
    cell_numbers = np.unique(mask)
    print("cell_numbers",cell_numbers)

    for cell_number in cell_numbers:    # loop for each segmentation mask
            print(cell_number)
            if cell_number == 0:
                continue 
        
            # identify whether the cell is neuron or astrocyte
            a=0  # NeuN channel: t0_c0; a for t, b for c
            b=0
            coords = get_pixel_coordinates(mask, cell_number)
            cropped_image = crop_to_fit(mask, coords, x, a, b, cell_number)
            is_neuron = neuron_or_not(cropped_image, threshold = 330)     # use Neun threshold optimized for actual image batch
            if is_neuron == True:
                # print("is neuron")           
                outdir = "./test/segmented_neurons/"    # set image output directory to the folder for neurons
                # z_series_list.append(x)
                cell_list.append(cell_number)
                cell_type_list.append('neuron')
                    
            else:
                a=5   # GFAP channel: t5_c0
                b=0
                
                coords = get_pixel_coordinates(mask, cell_number)
                expanded_coords = expand_coordinates(coords, padding)
                cropped_image2 = crop_to_fit_dilate(mask, expanded_coords, x, a, b, cell_number, padding)
                is_glial = glial_or_not_n_pixels(cropped_image2, 3000)       # intensity threshold set based on image intensity
                if is_glial == True:
                    # print("is glial")
                    outdir = "./test/segmented_astrocytes/"
                    # z_series_list.append(x)
                    cell_list.append(cell_number)
                    cell_type_list.append('astrocyte')
    
            count = 0
            if is_neuron or is_glial:    # only extract neuron and astrocyte images
                while count < 7:         # loop through all cycles for all protein images. Some color channels are empty.
                    if count<4:          # the first 4 cycle has 3 color channels of protein images
                        for a in range(0,4):
                            for b in range(0,3):
                                cropped_image = crop_to_fit(mask, coords, x, a, b, cell_number)
                                save_nuclei(cropped_image, outdir, x, a, b, cell_number)
                            count +=1
                            # print('Count:', count)
                    elif count == 4:
                        for a in range(4,6):   # Read the first (640 nm) and 3rd (488 nm) color channels in Cycle 5 & 6
                            for b in (0,2):
                                cropped_image = crop_to_fit(mask, coords, x, a, b, cell_number)
                                save_nuclei(cropped_image, outdir, x, a, b, cell_number)
                            count +=1
                            # print("Count:",count)
                    elif count<7:
                        a = 6              
                        for b in (2,3):   # Read the 3rd (488 nm) color channels in Cycle 7 
                            cropped_image = crop_to_fit(mask, coords, x, a, b, cell_number)
                            save_nuclei(cropped_image, outdir, x, a, b, cell_number)
                            count +=1
                            # print("Count:",count)
            else:
                cell_list.append(cell_number)
                cell_type_list.append('none')
                # print("Neither neuron or glial cell.")

    outfolder = "cells_list/"      # list of identification labels (neuron, astrocyte, or none) for each nuclear mask
    out_prefix = 'z'+str(x)
    if not os.path.exists(outfolder):
        os.makedirs(outfolder)
        
    out_report = os.path.join(outfolder,out_prefix+'.txt')
    with open(out_report, 'w') as f:
        for a, b in zip(cell_list, cell_type_list):
            f.write(f"{a}\t{b}\n")


0
cell_numbers [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199]
0
1
Neither neuron or glial cell.
2
Neither neuron or glial cell.
3
is glial
4
Neither neuron or glial cell.
5
is glial
6
is glial
7
Neither neuron or glial cell.
8
Neither

KeyboardInterrupt: 

## End